# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [5]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [6]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [7]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [11]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [13]:
# Getting the data from the top 10 expensive neighbourhoods
top_10_df = to_data.groupby("neighbourhood").mean()
top_10_df = top_10_df.sort_values("average_house_value", ascending=False).head(10)
top_10_df = top_10_df.reset_index()
top_10_df

# Calculate the mean number of dwelling types units per year
avg_house_val = to_data["average_house_value"].groupby([to_data.index]).mean()
avg_house_val.head()
# Calculate the average monthly shelter costs for owned and rented dwellings
avg_shelter_owned = to_data["shelter_costs_owned"].groupby([to_data.index]).mean()
avg_shelter_rented = to_data["shelter_costs_rented"].groupby([to_data.index]).mean()
avg_shelter_owned_df = pd.DataFrame(avg_shelter_owned)
avg_shelter_rented_df = pd.DataFrame(avg_shelter_rented)

### Panel Visualization Functions

In [14]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
    mean_neighbourhoods = to_data.groupby("neighbourhood").mean()
    mean_neighbourhoods = mean_neighbourhoods.reset_index()
    
    top_10 = to_data.groupby("neighbourhood").mean()
    top_10 = top_10.sort_values("average_house_value", ascending=False).head(10)
    top_10 = top_10.reset_index()
    values_and_locations_df = pd.concat([
                                df_neighbourhood_locations,
                                mean_neighbourhoods["average_house_value"]
                               ], axis=1).dropna()
    values_and_locations_df.head()
    
    px.set_mapbox_access_token(mapbox_token)
    
    map = px.scatter_mapbox(
        values_and_locations_df,
        lat="lat",
        lon="lon",
        color_continuous_scale=px.colors.cyclical.IceFire,
        size_max=15,
        zoom=3,
        width=1000,
        hover_name="neighbourhood",
        title="Average Vales Per Neighbourhood in Toronto",
)

    
    plotly_panel = pn.pane.Plotly(map)
    plotly_panel._updates = True
    return plotly_panel
     


def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    fig = data.hvplot.bar(title=title, xlabel=xlabel, ylabel=ylabel, rot=90, height=400, width=500, color=color)
    return fig
   

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    
    # YOUR CODE HERE!

def average_house_value():
    """Average house values per year."""
    
    average_house_values = to_data.groupby('year').mean()
    
    average_house_value_per_year_plot = plt.figure()
    min = average_house_values.min()['average_house_values']
    max = average_house_values.max()['average_house_values']

    average_house_values.plot.bar(ylim =(min-2000, max+2000),title="Average House Value per year in Toronto", figsize=(12,8))


    plt.close(average_house_value_per_year_plot)
    
    
    return pn.pane.Matplotlib(average_house_value_per_year_plot)

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    mean_neighbourhoods = to_data.groupby("neighbourhood").mean()
    mean_neighbourhoods = mean_neighbourhoods.reset_index()
    mean_neighbourhoods.head()
    

def number_dwelling_types():
    """Number of dwelling types per year"""
    dwelling_types = to_data.groupby("year").sum()
    dwelling_types.dropna()
    

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    to_data_new = to_data.groupby([to_data.index, "neighbourhood"]).mean()
    to_hv = to_data_new["average_house_value"]
    to_hv_df = pd.DataFrame(to_hv).reset_index()
    to_hv_df.head()
    

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    top_10_df = to_data.groupby("neighbourhood").mean()
    top_10_df = top_10_df.sort_values("average_house_value", ascending=False).head(10)
    top_10_df = top_10_df.reset_index()
    top_10_df
    



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [22]:
# Create a Title for the Dashboard
title = pn.pane.Markdown(
    """
# Rental Analysis of Toronto from 2001 to 2016
""",
    width=800,
)
welcome = pn.pane.Markdown(
    """
This dashboard presents a visual analysis of the Toronto rental market.
"""
)
# Create a tab layout for the dashboard
tabs = pn.Tabs(
    ("Welcome", pn.Column(welcome, neighbourhood_map())),
    ("Yearly Market Analysis", pn.Row(to_hv_df(),dwelling_types(),average_house_values()),
    ("Neighborhood Analysis", pn.Column(top_10_df(), mean_neighbourhoods()),
    (
        "Parallel Plots Analysis",
        pn.Column(parallel_coordinates(), parallel_categories(), width=960))




SyntaxError: unexpected EOF while parsing (1708433386.py, line 22)

## Serve the Panel Dashboard

In [ ]:
dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()